# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pisco,-13.7000,-76.2167,20.03,77,0,9.26,PE,1698625242
1,1,waitangi,-43.9535,-176.5597,10.94,77,2,3.37,NZ,1698625242
2,2,bethel,41.3712,-73.4140,9.64,88,100,3.09,US,1698625152
3,3,cidreira,-30.1811,-50.2056,20.84,89,55,2.79,BR,1698625243
4,4,ancud,-41.8697,-73.8203,7.34,75,57,10.02,CL,1698625243


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    hover_cols=["City"],
    line_color="black", 
    color = "City" 
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#Filter ideal conditions: Max temperature less than 27 degrees Celsius, Max temperature greater than 21 degrees Celsius,
#0% cloudiness, Wind speed less than 12.5 m/s

indeal_conditions_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Cloudiness']== 0) & (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
clean_data_df = indeal_conditions_df.dropna()

# Display sample data
clean_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
138,138,awjilah,29.1081,21.2869,21.53,43,0,2.85,LY,1698625264
185,185,tazacorte,28.6290,-17.9293,22.54,80,0,3.13,ES,1698625275
192,192,sultanah,24.4926,39.5857,21.25,73,0,2.57,SA,1698625277
196,196,huilong,31.8111,121.6550,22.18,69,0,3.44,CN,1698625278
204,204,salinas,36.6777,-121.6555,25.35,34,0,2.57,US,1698625034
241,241,murzuq,25.9155,13.9184,22.54,26,0,3.99,LY,1698625288
260,260,lincolnton,35.4737,-81.2545,22.83,74,0,0.00,US,1698625185
272,272,tsiombe,-25.3000,45.4833,21.17,83,0,2.21,MG,1698625295
301,301,ferkessedougou,9.6000,-5.2000,25.53,61,0,1.46,CI,1698625301
323,323,dana point,33.4670,-117.6981,26.08,15,0,2.68,US,1698625304


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
138,awjilah,LY,29.1081,21.2869,43,
185,tazacorte,ES,28.6290,-17.9293,80,
192,sultanah,SA,24.4926,39.5857,73,
196,huilong,CN,31.8111,121.6550,69,
204,salinas,US,36.6777,-121.6555,34,
241,murzuq,LY,25.9155,13.9184,26,
260,lincolnton,US,35.4737,-81.2545,74,
272,tsiombe,MG,-25.3000,45.4833,83,
301,ferkessedougou,CI,9.6000,-5.2000,61,
323,dana point,US,33.4670,-117.6981,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "accommodation.hotel", 
    "limit":20,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
awjilah - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
sultanah - nearest hotel: فندق جلنار
huilong - nearest hotel: 先豪国际酒店
salinas - nearest hotel: The Traveler's Hotel
murzuq - nearest hotel: فندق باريس - مرزق
lincolnton - nearest hotel: Days Inn by Wyndham Lincolnton
tsiombe - nearest hotel: No hotel found
ferkessedougou - nearest hotel: Ferké Hôtel Iroko
dana point - nearest hotel: Laguna Cliffs Marriott Resort & Spa
urangan - nearest hotel: Ramada
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
tafi viejo - nearest hotel: Casa Calchaqui
bobo-dioulasso - nearest hotel: Teria
fort bragg - nearest hotel: Airborne Inn Lodging
laguna - nearest hotel: Holiday Inn Express & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
138,awjilah,LY,29.1081,21.2869,43,No hotel found
185,tazacorte,ES,28.6290,-17.9293,80,App Leyma
192,sultanah,SA,24.4926,39.5857,73,فندق جلنار
196,huilong,CN,31.8111,121.6550,69,先豪国际酒店
204,salinas,US,36.6777,-121.6555,34,The Traveler's Hotel
241,murzuq,LY,25.9155,13.9184,26,فندق باريس - مرزق
260,lincolnton,US,35.4737,-81.2545,74,Days Inn by Wyndham Lincolnton
272,tsiombe,MG,-25.3000,45.4833,83,No hotel found
301,ferkessedougou,CI,9.6000,-5.2000,61,Ferké Hôtel Iroko
323,dana point,US,33.4670,-117.6981,15,Laguna Cliffs Marriott Resort & Spa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    hover_cols=["City","Hotel Name","Country"],
    line_color="black", 
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)